In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb

from sklearn.metrics import r2_score

In [12]:
data = pd.read_csv("drive/My Drive/Colab Notebooks/NYC_house.csv")

In [13]:
data

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,10009,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,10009,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84543,8409,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,,B9,37 QUAIL LANE,,10309,2,0,2,2400,2575,1998,1,B9,450000,2016-11-28 00:00:00
84544,8410,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,,B9,32 PHEASANT LANE,,10309,2,0,2,2498,2377,1998,1,B9,550000,2017-04-21 00:00:00
84545,8411,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,,B2,49 PITNEY AVENUE,,10309,2,0,2,4000,1496,1925,1,B2,460000,2017-07-05 00:00:00
84546,8412,5,WOODROW,22 STORE BUILDINGS,4,7100,28,,K6,2730 ARTHUR KILL ROAD,,10309,0,7,7,208033,64117,2001,4,K6,11693337,2016-12-21 00:00:00


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      84548 non-null  int64 
 1   BOROUGH                         84548 non-null  int64 
 2   NEIGHBORHOOD                    84548 non-null  object
 3   BUILDING CLASS CATEGORY         84548 non-null  object
 4   TAX CLASS AT PRESENT            84548 non-null  object
 5   BLOCK                           84548 non-null  int64 
 6   LOT                             84548 non-null  int64 
 7   EASE-MENT                       84548 non-null  object
 8   BUILDING CLASS AT PRESENT       84548 non-null  object
 9   ADDRESS                         84548 non-null  object
 10  APARTMENT NUMBER                84548 non-null  object
 11  ZIP CODE                        84548 non-null  int64 
 12  RESIDENTIAL UNITS               84548 non-null

In [15]:
data.isna().sum()

Unnamed: 0                        0
BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING CLASS CATEGORY           0
TAX CLASS AT PRESENT              0
BLOCK                             0
LOT                               0
EASE-MENT                         0
BUILDING CLASS AT PRESENT         0
ADDRESS                           0
APARTMENT NUMBER                  0
ZIP CODE                          0
RESIDENTIAL UNITS                 0
COMMERCIAL UNITS                  0
TOTAL UNITS                       0
LAND SQUARE FEET                  0
GROSS SQUARE FEET                 0
YEAR BUILT                        0
TAX CLASS AT TIME OF SALE         0
BUILDING CLASS AT TIME OF SALE    0
SALE PRICE                        0
SALE DATE                         0
dtype: int64

In [34]:
data['SALE PRICE'].unique()

array(['6625000', ' -  ', '3936272', ..., '408092', '11693337', '69300'],
      dtype=object)

In [89]:
def onehot_encode(df, columns, prefixes):
    df = df.copy()
    
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(df[column], prefix=prefix)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    
    return df

# Preprocessing

In [96]:
def preprocess_inputs(df):
  df = df.copy()
  df.columns = df.columns.str.lower().str.replace(' ', '_')
  df = df.rename(columns={"ease-ment": "easement"})

  df['sale_price'] = df['sale_price'].replace(' -  ', np.NaN).astype(np.float)
  #dropping the rows in sale_price column having missing value
  df = df.dropna(axis=0).reset_index(drop=True)

  df = df.drop(["unnamed:_0", "block", "lot", "easement", "address", "apartment_number"], axis=1)

  #fill all missing value with NaN
  df = df.replace(' -  ', np.NaN)

  #fill missing values with column mean
  for column in ["land_square_feet", "gross_square_feet"]:
    df[column] = df[column].astype(np.float)
    df[column] = df[column].fillna(df[column].mean())

  df['sale_date'] = pd.to_datetime(df['sale_date'])
  df['year']= df['sale_date'].apply(lambda x: x.year)
  df['month']= df['sale_date'].apply(lambda x: x.month)
  df['day']= df['sale_date'].apply(lambda x: x.day)

  df = df.drop('sale_date', axis=1)

  #make numerical and categorical columns in string columns
  for column in ["borough", "zip_code"]:
    df[column] = df[column].astype(str)

  #one hot encoding
  df = onehot_encode(
      df,
      columns=[
               'borough', 'zip_code', 'neighborhood', 'building_class_category',
               'tax_class_at_present', 'building_class_at_present', 'building_class_at_time_of_sale'
               ],
               prefixes=['bo', 'zc', 'ne', 'bc', 'tx', 'bp', 'bs']
               )

  #X and y
  X = df.drop("sale_price", axis=1)
  y = df['sale_price']

  scaler = StandardScaler()
  X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

  return X, y

In [97]:
X, y = preprocess_inputs(data)

In [98]:
X

,residential_units,commercial_units,total_units,land_square_feet,gross_square_feet,year_built,tax_class_at_time_of_sale,year,month,day,bo_1,bo_2,bo_3,bo_4,bo_5,zc_0,zc_10001,zc_10002,zc_10003,zc_10004,zc_10005,zc_10006,zc_10007,zc_10009,zc_10010,zc_10011,zc_10012,zc_10013,zc_10014,zc_10016,zc_10017,zc_10018,zc_10019,zc_10021,zc_10022,zc_10023,zc_10024,zc_10025,zc_10026,zc_10027,...,bs_R7,bs_R8,bs_R9,bs_RA,bs_RB,bs_RG,bs_RH,bs_RK,bs_RP,bs_RR,bs_RS,bs_RT,bs_RW,bs_S0,bs_S1,bs_S2,bs_S3,bs_S4,bs_S5,bs_S9,bs_T2,bs_U1,bs_V0,bs_V1,bs_V2,bs_V3,bs_V6,bs_V9,bs_W1,bs_W2,bs_W3,bs_W4,bs_W6,bs_W8,bs_W9,bs_Y1,bs_Y3,bs_Z0,bs_Z2,bs_Z9
0,0.213097,-0.018906,0.168314,-0.059242,0.113082,0.193234,0.464269,0.725650,0.126698,0.295594,1.961398,-0.334663,-0.723494,-0.598129,-0.306524,-0.106025,-0.047265,-0.059512,-0.092205,-0.034249,-0.04978,-0.048316,-0.063037,20.088532,-0.075817,-0.112584,-0.055769,-0.086519,-0.074858,-0.108209,-0.076007,-0.027268,-0.106369,-0.095612,-0.109752,-0.109218,-0.096747,-0.09431,-0.05021,-0.064616,...,-0.00378,-0.033616,-0.12718,-0.008453,-0.062116,-0.097646,-0.035076,-0.037446,-0.056665,-0.021051,-0.045564,-0.021388,-0.019645,-0.014145,-0.064728,-0.099273,-0.046034,-0.040921,-0.039313,-0.051061,-0.005346,-0.005346,-0.106575,-0.064057,-0.00378,-0.009259,-0.005346,-0.010692,-0.008453,-0.01363,-0.008453,-0.00756,-0.00378,-0.005346,-0.014145,-0.00378,-0.00378,-0.011954,-0.006547,-0.043635
1,0.556753,-0.018906,0.457734,-0.040280,0.127547,0.218191,0.464269,-1.378076,0.706273,0.749662,1.961398,-0.334663,-0.723494,-0.598129,-0.306524,-0.106025,-0.047265,-0.059512,-0.092205,-0.034249,-0.04978,-0.048316,-0.063037,20.088532,-0.075817,-0.112584,-0.055769,-0.086519,-0.074858,-0.108209,-0.076007,-0.027268,-0.106369,-0.095612,-0.109752,-0.109218,-0.096747,-0.09431,-0.05021,-0.064616,...,-0.00378,-0.033616,-0.12718,-0.008453,-0.062116,-0.097646,-0.035076,-0.037446,-0.056665,-0.021051,-0.045564,-0.021388,-0.019645,-0.014145,-0.064728,-0.099273,-0.046034,-0.040921,-0.039313,-0.051061,-0.005346,-0.005346,-0.106575,-0.064057,-0.00378,-0.009259,-0.005346,-0.010692,-0.008453,-0.01363,-0.008453,-0.00756,-0.00378,-0.005346,-0.014145,-0.00378,-0.00378,-0.011954,-0.006547,-0.043635
2,0.281829,-0.018906,0.226198,-0.037401,0.038510,0.193234,0.464269,-1.378076,1.285848,0.068561,1.961398,-0.334663,-0.723494,-0.598129,-0.306524,-0.106025,-0.047265,-0.059512,-0.092205,-0.034249,-0.04978,-0.048316,-0.063037,20.088532,-0.075817,-0.112584,-0.055769,-0.086519,-0.074858,-0.108209,-0.076007,-0.027268,-0.106369,-0.095612,-0.109752,-0.109218,-0.096747,-0.09431,-0.05021,-0.064616,...,-0.00378,-0.033616,-0.12718,-0.008453,-0.062116,-0.097646,-0.035076,-0.037446,-0.056665,-0.021051,-0.045564,-0.021388,-0.019645,-0.014145,-0.064728,-0.099273,-0.046034,-0.040921,-0.039313,-0.051061,-0.005346,-0.005346,-0.106575,-0.064057,-0.00378,-0.009259,-0.005346,-0.010692,-0.008453,-0.01363,-0.008453,-0.00756,-0.00378,-0.005346,-0.014145,-0.00378,-0.00378,-0.011954,-0.006547,-0.043635
3,0.419291,-0.018906,0.341966,-0.055770,0.022615,0.231630,0.464269,-1.378076,0.706273,0.749662,1.961398,-0.334663,-0.723494,-0.598129,-0.306524,-0.106025,-0.047265,-0.059512,-0.092205,-0.034249,-0.04978,-0.048316,-0.063037,20.088532,-0.075817,-0.112584,-0.055769,-0.086519,-0.074858,-0.108209,-0.076007,-0.027268,-0.106369,-0.095612,-0.109752,-0.109218,-0.096747,-0.09431,-0.05021,-0.064616,...,-0.00378,-0.033616,-0.12718,-0.008453,-0.062116,-0.097646,-0.035076,-0.037446,-0.056665,-0.021051,-0.045564,-0.021388,-0.019645,-0.014145,-0.064728,-0.099273,-0.046034,-0.040921,-0.039313,-0.051061,-0.005346,-0.005346,-0.106575,-0.064057,-0.00378,-0.009259,-0.005346,-0.010692,-0.008453,-0.01363,-0.008453,-0.00756,-0.00378,-0.005346,-0.014145,-0.00378,-0.00378,-0.011954,-0.006547,-0.043635
4,1.518989,-0.018906,1.268108,0.025508,0.606811,0.231630,0.464269,-1.378076,1.285848,-1.066608,1.961398,-0.334663,-0.723494,-0.598129,-0.306524,-0.106025,-0.047265,-0.059512,-0.092205,-0.034249,-0.04978,-0.048316,-0.063037,20.08

In [95]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69987 entries, 0 to 69986
Columns: 833 entries, residential_units to bs_Z9
dtypes: float64(2), int64(8), uint8(823)
memory usage: 60.3 MB


In [77]:
y

0         6625000.0
1         3936272.0
2         8000000.0
3         3192840.0
4        16232000.0
            ...    
69982      450000.0
69983      550000.0
69984      460000.0
69985    11693337.0
69986       69300.0
Name: sale_price, Length: 69987, dtype: float64

In [78]:
y.unique()

array([ 6625000.,  3936272.,  8000000., ...,   408092., 11693337.,
          69300.])

In [79]:
y.isna().sum()

0

In [80]:
print("Percentage of missing value is :", (y.isna().mean())*100)

Percentage of missing value is : 0.0


In [81]:
X.isna().sum()

borough                           0
neighborhood                      0
building_class_category           0
tax_class_at_present              0
building_class_at_present         0
zip_code                          0
residential_units                 0
commercial_units                  0
total_units                       0
land_square_feet                  0
gross_square_feet                 0
year_built                        0
tax_class_at_time_of_sale         0
building_class_at_time_of_sale    0
sale_date                         0
dtype: int64

#Training the model using XGBoost

In [101]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [102]:
params = {'learning_rate': 0.001, 'max_depth': 6, 'lambda': 0.01}

model = xgb.train(params, dtrain, num_boost_round=10000, evals=[(dval, 'eval')], early_stopping_rounds=10)

[0]	eval-rmse:6.59545e+06
Will train until eval-rmse hasn't improved in 10 rounds.
[1]	eval-rmse:6.59263e+06
[2]	eval-rmse:6.5899e+06
[3]	eval-rmse:6.58726e+06
[4]	eval-rmse:6.5847e+06
[5]	eval-rmse:6.58222e+06
[6]	eval-rmse:6.57984e+06
[7]	eval-rmse:6.57754e+06
[8]	eval-rmse:6.57531e+06
[9]	eval-rmse:6.57318e+06
[10]	eval-rmse:6.57113e+06
[11]	eval-rmse:6.56917e+06
[12]	eval-rmse:6.56729e+06
[13]	eval-rmse:6.5655e+06
[14]	eval-rmse:6.56379e+06
[15]	eval-rmse:6.56217e+06
[16]	eval-rmse:6.56063e+06
[17]	eval-rmse:6.55917e+06
[18]	eval-rmse:6.5578e+06
[19]	eval-rmse:6.5565e+06
[20]	eval-rmse:6.5553e+06
[21]	eval-rmse:6.55417e+06
[22]	eval-rmse:6.55313e+06
[23]	eval-rmse:6.55217e+06
[24]	eval-rmse:6.55128e+06
[25]	eval-rmse:6.55049e+06
[26]	eval-rmse:6.54978e+06
[27]	eval-rmse:6.54913e+06
[28]	eval-rmse:6.54858e+06
[29]	eval-rmse:6.54811e+06
[30]	eval-rmse:6.54772e+06
[31]	eval-rmse:6.5474e+06
[32]	eval-rmse:6.54718e+06
[33]	eval-rmse:6.54708e+06
[34]	eval-rmse:6.54701e+06
[35]	eval-rmse:

In [103]:
y_true = np.array(y_test)
y_pred = model.predict(dtest)

In [104]:
print("Model R^2 Score: {:.4f}".format(r2_score(y_true, y_pred)))

Model R^2 Score: -0.0156
